In [7]:
# Import required libraries
import numpy as np
import pandas as pd 
# For cross-validation
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
# For fitting the model
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LogisticRegression
# For model evaluation
from sklearn.metrics import roc_auc_score

# Save the model
import pickle


## Data Preparation

In [8]:
df = pd.read_csv('../data/Telco_Customer_Churn.csv')
df.columns = df.columns.str.lower().str.replace(' ', '_')
categorical_cols = list(df.dtypes[df.dtypes == 'object'].index)

for col in categorical_cols:
    df[col] = df[col].str.lower().str.replace(' ', '_')

df.totalcharges = pd.to_numeric(df.totalcharges, errors='coerce')
df.totalcharges = df.totalcharges.fillna(df.totalcharges.mean())
df.churn = (df.churn == 'yes').astype(int)

In [9]:
df_train_full, df_test = train_test_split(df, test_size=0.2, random_state=1)

In [10]:
numerical = ['monthlycharges', 'tenure', 'totalcharges']

categorical = ['gender',
               'seniorcitizen', 
               'partner', 
               'dependents', 
               'phoneservice', 
               'multiplelines', 
               'internetservice', 
               'onlinesecurity', 
               'onlinebackup', 
               'deviceprotection', 
               'techsupport', 
               'streamingtv', 
               'streamingmovies', 
               'contract', 
               'paperlessbilling', 
               'paymentmethod']

In [11]:
def train(df_train, y_train, C=1.0):
    dicts = df_train[categorical + numerical].to_dict(orient='records')
    dv = DictVectorizer(sparse=False)
    X_train = dv.fit_transform(dicts)

    model = LogisticRegression(C=C, max_iter=10000)
    model.fit(X_train, y_train)

    return dv, model 

In [12]:
def predict(dv, df, model):
    dicts = df[categorical + numerical].to_dict(orient='records')
    X = dv.transform(dicts)
    y_pred = model.predict_proba(X)[:, 1]

    return y_pred

In [13]:
C=1.0
n_split = 5

In [14]:
kfold = KFold(n_splits=n_split, shuffle=True, random_state=1)

scores = []

for train_index, validation_index in kfold.split(df_train_full):
    df_train = df_train_full.iloc[train_index]
    df_validation = df_train_full.iloc[validation_index]

    y_train = df_train.churn
    y_val = df_validation.churn

    dv, model = train(df_train, y_train, C=C)
    y_pred = predict(dv, df_validation, model)

    auc = roc_auc_score(y_val, y_pred)
    scores.append(auc)
print('C=%s %.3f +- %.3f'%(C, np.mean(scores), np.std(scores)))

C=1.0 0.842 +- 0.007


In [15]:
dv, model = train(df_train_full, df_train_full.churn.values, C=1.0)
y_pred = predict(dv, df_test, model)
y_test = df_test.churn.values
auc = roc_auc_score(y_test, y_pred)
auc

0.8583084164797903

In [16]:
output_file = f'model_C={C}.bin'

In [17]:
with open(output_file, 'wb') as f_out:
    pickle.dump((dv,model), f_out)

## Load the model

In [18]:
# input_file = 'model_C=1.0.bin'
# with open(input_file, 'rb') as f_in:
#     dv,model = pickle.load(f_in)